In [2]:

import clean_data
from get_sample_data import get_data_from_mongo
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from pymongo import MongoClient
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
import meetup_pipeline
from sklearn.preprocessing import LabelBinarizer


In [3]:
mc = MongoClient()
db = mc['meetups']
events = db['past_3years_meetup_events']
groups = db['groups']
ed = db['events_data']

In [5]:
shard = get_data_from_mongo(0.5)

In [6]:
df = pd.DataFrame(shard)

In [3]:
df =pd.read_csv('data.gzip', compression='gzip')
df = clean_data.drop_null_labels(df)

/home/cb/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [26]:
basic_set = df.loc[:, ['plain_text_no_images_description', 
                       'name', 'visibility', 'waitlist_count',
                       'yes_rsvp_count', 'description']]

In [27]:
basic_set = clean_data.drop_null_labels(basic_set)

In [28]:
basic_set = clean_data.fill_nulls(basic_set)

In [29]:
basic_set = X_train.dropna(axis=0)

In [30]:
basic_set['interest'] = basic_set['waitlist_count'] + basic_set['yes_rsvp_count']

/home/cb/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [31]:
y = basic_set.pop('interest')

In [32]:
X_train, X_test, y_train, y_test = train_test_split(basic_set, y)

In [33]:
tfidf_desc = TfidfVectorizer(stop_words='english',max_df=0.99, min_df=.01)

In [34]:
train_desc_vec = tfidf_desc.fit_transform(X_train['plain_text_no_images_description'])
test_desc_vec = tfidf_desc.transform(X_test['plain_text_no_images_description'])

In [35]:
tfidf_name = TfidfVectorizer(stop_words='english', max_df=0.99, min_df=.01)

In [36]:
train_name_vec = tfidf_name.fit_transform(X_train['name'])
test_name_vec = tfidf_name.transform(X_test['name'])

In [37]:
X_train_formatted = pd.concat([X_train.loc[:, ['has_waitlist', 'visibility']].reset_index(drop=True),
           pd.DataFrame(train_desc_vec.toarray()),
           pd.DataFrame(train_name_vec.toarray())], axis=1)

/home/cb/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [149]:
X_test_formatted = pd.concat([X_test.loc[:, ['has_waitlist', 'visibility']].reset_index(drop=True),
           pd.DataFrame(test_desc_vec.toarray()),
           pd.DataFrame(test_name_vec.toarray())], axis=1)

In [8]:
rf = RandomForestRegressor()

In [39]:
rf.fit(X_train_formatted, y_train)

ValueError: could not convert string to float: 'public_limited'

In [150]:
rf.score(X_test_formatted, y_test)

0.3004355422142446

In [148]:
X_train_formatted.shape

(116519, 100)

In [140]:
len(X_train.loc[:, ['has_waitlist', 'visibility']])

116519

In [134]:
train_name_vec.shape

(116519, 1)

In [135]:
train_desc_vec.shape

(116519, 97)

In [152]:
tfidf_name.get_feature_names()

['group']

In [14]:
okay = meetup_pipeline.visibility.fit_transform(test)

In [15]:
okay

array([[1],
       [1],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [23]:
meetup_pipeline.name_tfidf.transform(X_train)

<116519x24650 sparse matrix of type '<class 'numpy.float64'>'
	with 652758 stored elements in Compressed Sparse Row format>

In [21]:
Out[19].shape

(116519, 1)

In [21]:
X_train.isnull().sum()

plain_text_no_images_description    0
name                                1
visibility                          0
waitlist_count                      0
yes_rsvp_count                      0
description                         0
dtype: int64

In [30]:
test['visibility'].unique()

array(['public', 'public_limited'], dtype=object)

In [32]:
lb = LabelBinarizer()

In [43]:
lb.fit_transform(okay)

array([[0],
       [0],
       [0],
       ...,
       [0],
       [1],
       [0]])

In [45]:
okay

array([['public'],
       ['public'],
       ['public'],
       ...,
       ['public'],
       ['public_limited'],
       ['public']], dtype=object)

In [16]:
okay

array(['public_limited', 'public', 'public', ..., 'public',
       'public_limited', 'public'], dtype=object)

In [4]:
okay =meetup_pipeline.interest.fit_transform(df)

In [10]:
okay

<155386x127173 sparse matrix of type '<class 'numpy.float64'>'
	with 11198398 stored elements in Compressed Sparse Row format>

In [ ]:
okay

In [ ]:
meetup_pipeline.model.fit_transform(df, okay)

In [26]:
meetup_pipeline.has_waitlist.fit_transform(df).shape

(155359, 1)

In [32]:
meetup_pipeline.name_tfidf.fit_transform(df).shape

(155359, 56)

In [7]:
x = meetup_pipeline.meetup_union.fit_transform(df)

In [9]:
rf.fit(x, okay)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [11]:
meetup_pipeline.meetup_model.fit_transform(df, okay)

RFRWrapper(bootstrap=True, criterion='mse', max_depth=None,
      max_features='auto', max_leaf_nodes=None, min_impurity_decrease=0.0,
      min_impurity_split=None, min_samples_leaf=1, min_samples_split=2,
      min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
      oob_score=False, random_state=None, verbose=0, warm_start=False)